# JavaScript Graphics in Jupyter
* [Introduction](#introduction)
* [General Approach](#general_approach)
* [Libraries and Packages](#libraries_packages)
    * [math.js](#mathjs)
    * [Apache ECharts](#echarts)
    * [jQuery](#jquery)
* [Demonstration of Methodology](#methodology_example)
* [JavaScript vs Python Performance Comparison](#js_python_comparison)
    * [JavaScript Performance Demo](#js_performance_demo)
    * [Python Performance Demo](#python_performance_demo)
* [Browser Support](#browser_support)
* [Taylor Series Approximation](#taylor_series_application)
***

<a class="anchor" id="introduction"></a>
## Introduction

Interactive widgets that provide graphical representation of data and can be modified according to user controlled variables are an invaluable pedagogical tool. Jupyter notebooks offers an excellent hosting platform for for such tools, and we have seen static graphics used effectively within this environment.

Jupyter offers a limited degree of functionality for interactive GUI controls via the ipywidgets.interact package, however this package is limited in its scope of use cases and offers little control over UI design. Furthermore, user driven events are handled by the underlying notebook kernel on the server, and require requests to exchanged between the client browser and the server. For users with low Internet bandwidth, applications and widgets that require frequent updates can become unresponsive. In a case in which a simple plot of a polynomial (plotted via the matplotlib library) was modified based on a slider bar, we noted significant lag between the dragging of the slider bar and the rendering of the updated graphic.

HTML and CSS may be used to place and style UI elements within a Jupyter notebook cell.

JavaScript may be used to render graphics natively within the browser, bypassing the need to call into the notebook kernel with each user initiated event. Much like a single-page application (SPA), the application code is downloaded once. User driven events do not trigger a request to the server. Initial trials subjectively show a significant reduction in lag, resulting in a much more responsive application.

<a class="anchor" id="motivation"></a>
## General Approach

Jupyter has some functionality that they refer to as "magics" which allow a cell to execute in a subkernel that's different from the rest of the notebook. We will be using HTML and JavaScript subkernels.

With HTML cells we first create UI elements with "id" attributes set. 

We then create Javascript cells that reference these HTML element id's when scripting.

<a class="anchor" id="libraries_packages"></a>
## Libraries and Packages

With the exception of jQuery, all JavaScript libraries are linked using Content Distribution Networks (CDN) rather than using a copy installed on the server. The advantage to this approach is that it doesn't require relative path information when referencing the library.

<a class="anchor" id="mathjs"></a>
### math.js

- General: https://mathjs.org/
- There seem to be a lot of ways to parse/evaluate expressions. It's pretty flexible, which is nice.

<a class="anchor" id="echarts"></a>
### Apache ECharts

- General: https://echarts.apache.org/
- Chart styling: https://echarts.apache.org/en/option.html
- Good:
    - Nice documentation (see "Bad")... 
    - Mature
    - Widely adopted; used by a few large organizations (notably GitLab)
    - Low risk of discontinued support
    - Highly adaptable
    - Very pretty, really nice interactivity, graphics can display dense, rich information.
    - Non-American primary development could have the benefit of communicating information in such a way that
    -     foreign attributes or ites 
- Bad:
    - Nice documentation... examples are not flushed out or explained much.
    - No help forum or contact info
    - Some fairly common use cases for mathematical plotting are missing.

<a class="anchor" id="jquery"></a>
### jQuery

jQuery is already installed, so we can use the $ syntax to conveniently access HTML elements by id.

<a class="anchor" id="methodology_example"></a>
## Demonstration of Methodology: Wave Amplitude Application

This example shows how to make an HTML skeleton container, load JavaScript libraries, and populate the container.

In this application the user may use a slider control to adjust the amplitude of a sine function graph.

*The snippet below shows creates a "skeleton" template that we will fill out later with JavaScript:*

In [38]:
%%html

<div style="width:fit-content;">
    <div id="chart_sine"></div>
    <div style="display:flex;justify-content:center;width:fit-content;margin:0 auto;">
        <span style="font-weight:bold;margin-right:20px;">Amplitude:</span>
        <div id="chart_sine_slider" style="width:300px;margin:auto;"></div>
        <span id="amplitude" style="font-weight:bold;margin-left:20px;">(1.0)</span>
    </div>
</div>

In [39]:
%%javascript

// *Note that we are using arrow functions, (input) => output, in several places.
// Normal functions can be used if preferred.

// Any required libraries can be included with the "require" function call. 
// This is similar to an import statement.
// Syntax can be copied from this example. 
require([
    'https://cdnjs.cloudflare.com/ajax/libs/echarts/5.4.2/echarts.min.js',
    'https://cdnjs.cloudflare.com/ajax/libs/mathjs/11.8.2/math.min.js',
], (echarts,math) => {
    // The libraries above are assigned an alias, and can then be used in the anonymous function below.
    
    // This generates and returns a series of points (2d array). We've chosen the sine function for this example.
    // This function has been defined in the require
    function generateData(scalar) {
        let data = [];
        for (let i = -10; i <= 10; i += 0.1) {
            data.push([i, 1.0 * scalar * math.sin(i)]);
        }
        return data;
    }
    
    /* ******************************************************************************
    * BUILD THE CHART
    * ******************************************************************************/
    
    // Initialize the chart object using the empty HTML element from above.
    const myChart = echarts.init(document.getElementById('chart_sine'), 
        null, 
        {
            width: 600,
            height: 400
        });
    
    // Set the chart options.
    // See: https://echarts.apache.org/en/option.html
    let option = {
        animation: false,
        xAxis: {
            min: 'datamin',
            max: 'datamax',
            splitLine: {show: false}, // Hide the grid lines
        },
        yAxis: {
            min: -1.5,
            max: 1.5
        },
        series: {
            type: 'line',
            data: generateData(1.0),
            showSymbol: false,
        },
        title: {
            left: 'center',
            text: 'Sine Wave with Adjustable Amplitude'
        },
        tooltip: {
            trigger: 'axis'
        },
    };

    // If the chart option exists (is should - this is a sanity check), display the chart.
    option && myChart.setOption(option);
    
    /* ******************************************************************************
    * ADD THE SLIDER
    * ******************************************************************************/
    
    // *Note that the slider code must be in the same cell that has the dataset so that the data variable
    // is in scope.
    
    // jQuery functions are wrapped in this dollar sign syntax.
    $(() => {        
        
        // Create the slider.
        $("#chart_sine_slider").slider({
            min: 0,
            max: 100,
            step: 0.1,
            value: 100,
            // The slide event is triggered on every mouse move during the slide. If we hit lag on more complex models with
            // a slider we can switch to the change event, which won't fire until the user lets go of the mouse button.
            slide: (event, ui) => {
                // In this case we're scaling the amplitude based on the normalized slider value: 
                var sliderVal = ui.value / 100.0;
                sliderVal = math.round(sliderVal,2);
                
                // Assign new data to the chart object and redraw.
                option.series.data = generateData(sliderVal);
                option && myChart.setOption(option, true); // second parameter must be true to cause a redraw in some situations. Ref: https://echarts.apache.org/en/api.html#echartsInstance.setOption

                // Set the amplitude label to show the value to the user.
                $("#amplitude").html('(' + sliderVal + ')' )
            }
        });
    });
});

<IPython.core.display.Javascript object>

<a class="anchor" id="js_python_comparison"></a>
## JavaScript vs Python Performance Comparison

Here we offer the same functionality using the two technologies, one with JS and the other Python.

You may drag the slider back and forth repeatedly for each and note the time between the slider drag and the rendering of the image.

<a class="anchor" id="js_performance_demo"></a>
### JavaScript Performance Demo

The Javascript library paints the graph on an HTML canvas element. It does not render or use any image files.

In [14]:
%%html

<div style="width:fit-content;">
    <div id="chart_comparison"></div>
    <div style="display:flex;justify-content:center;width:fit-content;margin:0 auto;">
        <span style="font-weight:bold;margin-right:20px;">y2:</span>
        <div id="chart_comparison_slider" style="width:300px;margin:auto;"></div>
        <span id="chart_comparison_slider_value" style="font-weight:bold;margin-left:20px;">(10)</span>
    </div>
</div>

In [36]:
%%javascript

/* ****************************************************************
* SET THE GRID SIZE TO SCALE
* ****************************************************************/
const MAX_SIZE = 400;

const gridDims = {
    x: { min: 0, max: 12 },
    y: { min: 0, max: 12 },
}

// There's no option to have the x and y axis on the same scale, so we have to do this trick.
// By default it will fit the parent container.
function getGridWidthHeight() {
    const rangeX = gridDims.x.max - gridDims.x.min;
    const rangeY = gridDims.y.max - gridDims.y.min;
    
    let dims = {width: MAX_SIZE, height: MAX_SIZE}  
    if (rangeX > rangeY) {
        dims.height = MAX_SIZE * rangeY / rangeX;
    } else {
        dims.width = MAX_SIZE * rangeX / rangeY;
    }
    
    return dims;
}

/* ****************************************************************
* PLOT
* ****************************************************************/
require([
    'https://cdnjs.cloudflare.com/ajax/libs/echarts/5.4.2/echarts.min.js'
], (echarts) => {
    
    const gridSize = getGridWidthHeight();
    
    const myChart = echarts.init(document.getElementById('chart_comparison'), 
        null, 
        {
            width: gridSize.width,
            height: gridSize.height
        });
    
    let option = {
        animation: false,
        xAxis: {
            min: gridDims.x.min,
            max: gridDims.x.max,
            splitLine: {show: false}, // Hide the grid lines
        },
        yAxis: {
            min: gridDims.y.min,
            max: gridDims.y.max,
            splitLine: {show: false}, // Hide the grid lines
        },
        series: {
            data: [[0,0],[10,10]],
            type: 'line',
        },
        grid: {
            left: '50', // margin
        }
    };

    option && myChart.setOption(option);
    
     $(() => {
        $("#chart_comparison_slider").slider({
            min: 0,
            max: 10,
            step: 1,
            value: 10,
            slide: (event, ui) => {
                const sliderVal = ui.value;
                option.series.data[1][1] = sliderVal;
                option && myChart.setOption(option, true);
                $("#chart_comparison_slider_value").html('(' + sliderVal + ')' )
            }
        });
     });
});

<IPython.core.display.Javascript object>

<a class="anchor" id="python_performance_demo"></a>
###  Python Performance Demo

As the slider passes each integer the browser initiates a request to the server. The server passes back an image that's roughly 8.5 KB.

In [16]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact

# Receive user input and update the variables
@interact(
    y2 = widgets.IntSlider(value=10, min=0, max=10)
)

# Plot a straight line that goes through both points
def straight_line(y2):
    x1 = 0
    y1 = 0
    x2 = 10
    
    x = [x1,x2]
    y = [y1,y2]

    plt.plot(x, y)
    ax = plt.gca()
    ax.set_xlim(0, 12)
    ax.set_ylim(0, 12)
    ax.set_xticks( range(13) )
    ax.set_yticks( range(13) )
    ax.plot(x1, y1, color='#006DB3', marker='o')                  # draw the first point, P1
    ax.plot(x2, y2, color='#006DB3', marker='o')                  # draw the second point, P2
    

interactive(children=(IntSlider(value=10, description='y2', max=10), Output()), _dom_classes=('widget-interact…

<a class="anchor" id="browser_support"></a>
## Browser Support

The described HTML/JavaScript injection approach has been tested in the current versions following browsers as of 7/6/2023:


* Chrome (Blink, V8)
* Edge (Blink, Chakra)
* _TODO: Firefox (Gecko, SpiderMonkey) It will be much easier to test support when we deploy the repo to the test environment._
* _TODO: Safari (WebKit, JavaScriptCore) It will be much easier to test support when we deploy the repo to the test environment._

<a class="anchor" id="taylor_series_application"></a>
## Taylor Series Approximation

_TODO: I'm assuming this is going to get moved to the Taylor notebook. Waiting to here back from collaborators._

Gonna start with Maclaurin... It's easier. We can add a shift later if we want.

In [17]:
%%html

<!-- I'm tired of having to scroll up every time I make a change, so I'm dropping this in below the JS code temporarily
<div style="width:fit-content;">
    <div id="chart_taylor"></div>
    <div style="display:flex;justify-content:center;width:fit-content;margin:0 auto;">
        <span style="font-weight:bold;margin-right:20px;">Terms:</span>
        <div id="chart_taylor_slider" style="width:300px;margin:auto;"></div>
        <span id="term_count" style="font-weight:bold;margin-left:20px;">()</span>
    </div>
</div>
-->

In [71]:
%%javascript

// TODO: Try to plot multiple series against same x values. No need to take up memory space. with these
// TODO: Add "a" parameter for Taylor.
// TODO: The library can't render TeX, or any kind of superscript, so maybe put it in a section header above the graph?

// Closest eCharts example: https://echarts.apache.org/examples/en/editor.html?c=line-stack

/* ****************************************************************
* GRAPH SIZING OPTIONS
* ****************************************************************/
// TODO: extract this section to its own JS module/file. It can be reused.

const MAX_SIZE = 600;

const TICKS_PER_UNIT = 0.1;

const gridDims = {
    x: { min: 0, max: 5 },
    y: { min: 0, max: 10 },
}

// There's no option to have the x and y axis on the same scale, so we have to do this trick.
// By default it will fit the parent container.
function getGridWidthHeight() {
    const rangeX = gridDims.x.max - gridDims.x.min;
    const rangeY = gridDims.y.max - gridDims.y.min;
    
    let dims = {width: MAX_SIZE, height: MAX_SIZE}  
    if (rangeX > rangeY) {
        dims.height = MAX_SIZE * rangeY / rangeX;
    } else {
        dims.width = MAX_SIZE * rangeX / rangeY;
    }
    
    return dims;
}

/* ****************************************************************
* OTHER GRAPH OPTIONS
* ****************************************************************/
const axisOptions = {
    axisLabel: {show: true},
    axisTick: {show: true},
    //minorTick: {show: true},
    splitLine: {show: false}, // Hide the grid lines
    type: 'value',
}

/* ****************************************************************
* APPLICATION ENTRY POINT
* ****************************************************************/
require([
    'https://cdnjs.cloudflare.com/ajax/libs/echarts/5.4.2/echarts.min.js',
    'https://cdnjs.cloudflare.com/ajax/libs/mathjs/11.8.2/math.min.js',
], (echarts,math) => {
    
    /* ****************************************************************
    * DEFINE GRAPH FUNCTIONS
    * ****************************************************************/
    // TODO: extract this section to its own JS module/file. It's too big to go in here.
    // TODO: Add more functions later.
    
    const function_definitions = {
        e_pow_x: {
            fx: (x) => math.pow(math.e, x),
            terms: [
                (x) => 1.0, // 0th term
                (x) => x/1.0, // 1st term
                (x) => math.pow(x,2.0)/math.factorial(2), // 2nd term...
                (x) => math.pow(x,3.0)/math.factorial(3),
                (x) => math.pow(x,4.0)/math.factorial(4),
            ],
        }
    }
        
    function generateChartData(fn) {
        let series_index = 0;
        let xValues = [];
        let series_data = [];
        let cur_data = [];
        let sum = [];
        let i = 0;
        
        // Initialize the x axis and summation values.
        for (let x = gridDims.x.min; x <= gridDims.x.max; x += TICKS_PER_UNIT) {
            xValues.push(x);
            sum.push(0);
        }
        
        // Plot the function
        cur_data = [];
            
        for (let x = gridDims.x.min; x <= gridDims.x.max; x += TICKS_PER_UNIT) {
            const val = math.round(fn.fx(x),4);
            //cur_data.push([x, val]);
            cur_data.push(val);
        }

        series_data.push({
            name: 'f(x)', 
            data: cur_data, 
            showSymbol: false,
            stack: 'all',
            type: 'line', 
        });
        
        series_data.push({
            name: 'nth sum', 
            data: null, // To be set later
            showSymbol: false,
            stack: 'all',
            type: 'line', 
        });
        
        // Plot the terms
        fn.terms.forEach((f,fIndex) => {
            cur_data = [];
            
            // Reset the index for the summation array.
            i = 0;
            for (let x = gridDims.x.min; x <= gridDims.x.max; x += TICKS_PER_UNIT) {
                const val = math.round(f(x),4);
                //cur_data.push([x, val]);
                cur_data.push(val);
                sum[i] += val;
                i++;
            }
            
            series_data.push({
                name: 'n=' + fIndex,
                data: cur_data, 
                showSymbol: false,
                stack: 'all',
                type: 'line',                
            });
        });
        
        // Add in the summation array, transforming it into a 2d array.
        i = 0;
        cur_data = [];
        for (let x = gridDims.x.min; x <= gridDims.x.max; x += TICKS_PER_UNIT) {
            //cur_data.push([x, math.round(sum[i],2)]);
            cur_data.push(math.round(sum[i],2));
            i++;
        }
        
        // data[1] is the nth_sum series
        series_data[1].data = cur_data;

        return {
            series_data,
            xValues
        };
    }
    
    /* ******************************************************************************
    * BUILD THE CHART
    * ******************************************************************************/
    
    // TODO: Hardcoded to e_pow_x until we add a dropdown for function selection.
    const selectedFunction = function_definitions.e_pow_x;
    const chartData = generateChartData(function_definitions.e_pow_x);
    const gridSize = getGridWidthHeight();    
    const myChart = echarts.init(document.getElementById('chart_taylor'), 
        null, 
        {
            //width: gridSize.width,
            //height: gridSize.height
            // For e_pow_x it's easier to see the delta in the summation if we don't draw the axes to scale.
            // We'll have to include this as a property on the object for the other functions.
            width: MAX_SIZE,
            height: MAX_SIZE
        });
    
    // Debug
    console.log(chartData);
    
    // Set the chart options.
    // See: https://echarts.apache.org/en/option.html
    let option = {
        animation: false,
        grid: {
            left: '50', // margin
        },
        xAxis: {
            ...axisOptions,
            axislabel: {interval: 0},
            boundaryGap: false,
            data: chartData.xValues,
            max: gridDims.x.max,
            min: gridDims.x.min,
            name: 'x',
            type: 'category',
        },
        yAxis: {
            ...axisOptions,
            max: gridDims.y.max,
            min: gridDims.y.min,
            name: 'y',
            type: 'value'
        },
        series: chartData.series_data,
        title: {
            left: 'center',
            text: 'Taylor Series Approximation: e^x, a=0 (Maclauren Series)'
        },
        tooltip: {
            trigger: 'axis'
        },
    };

    // If the chart option exists (is should - this is a sanity check), display the chart.
    option && myChart.setOption(option);
    
    /* ******************************************************************************
    * UI Controls
    * ******************************************************************************/
    
    // jQuery functions are wrapped in this dollar sign syntax.
    $(() => {
        $("#chart_taylor_slider").slider({
            create: (event, ui) => {
                $("#term_count").html('(' + (selectedFunction.terms.length - 1).toString() + ')' )                
            },
            max: selectedFunction.terms.length - 1,
            min: 0,
            slide: (event, ui) => {
                // TODO: render the first n series. These are already calculated so rendering is fast.
                const sliderVal = ui.value;
                option.series = chartData.slice(0, sliderVal + 2); // The first two elements in the chartData array are f(x) and the summation. Always show these.
                option && myChart.setOption(option, true);
                $("#term_count").html('(' + sliderVal + ')' );
                
                // DEBUG
                console.log(option);
            },
            step: 1,
            value: selectedFunction.terms.length - 1,
        });
    });
});

<IPython.core.display.Javascript object>

In [62]:
%%html

<!-- DEBUG -->
<!-- This cell has to be run before the one above it. -->
<div style="width:fit-content;">
    <div id="chart_taylor"></div>
    <div style="display:flex;justify-content:center;width:fit-content;margin:0 auto;">
        <span style="font-weight:bold;margin-right:20px;">Terms:</span>
        <div id="chart_taylor_slider" style="width:300px;margin:auto;"></div>
        <span id="term_count" style="font-weight:bold;margin-left:20px;">()</span>
    </div>
</div>